In [1]:
import pandas as pd

In [2]:
#####

# README: This code turns x-y coordinate files into x-y-z files that can be used by Ovito

#####

# Change the filepath here to your x-y coordinates

file_path = '15217_building_coords'

file = open(file_path + '.csv')

coords = pd.read_csv(file, sep = ',', header = 'infer')

file.close()

coords

,x,y
0,-79.928838,40.432732
1,-79.931468,40.436137
2,-79.926850,40.432134
3,-79.912230,40.444182
4,-79.920037,40.418077
...,...,...
9727,-79.917187,40.430388
9728,-79.922373,40.429590
9729,-79.915851,40.447352
9730,-79.920934,40.431492


In [3]:
# Create the normalized dataframe

normalized_data = pd.DataFrame()

for letter in ['x', 'y']:
    
    oldMax = coords[letter].max()
    
    oldMin = coords[letter].min()
    
    abs_max = max(abs(oldMax), abs(oldMin))
    
    normalized_data[letter] = coords[letter].divide(abs_max)
    
    mean = normalized_data[letter].mean()

    normalized_data[letter] = normalized_data[letter].subtract(mean)

    normalized_data[letter] = normalized_data[letter].multiply(10000)

normalized_data

,x,y
0,-0.517051,0.078102
1,-0.846049,0.919888
2,-0.268364,-0.069735
3,1.560511,2.908776
4,0.583902,-3.544912
...,...,...
9727,0.940420,-0.501382
9728,0.291682,-0.698664
9729,1.107545,3.692464
9730,0.471692,-0.228451


In [4]:
normalized_data.insert(0, 'test', 'H')

normalized_data[''] = 0

normalized_data.loc[-1] = ['', '', '', '']

normalized_data.index = normalized_data.index + 1

normalized_data = normalized_data.sort_index()

r, c = normalized_data.shape

# Setting one of the 'z' values as 0.01 to prevent degeneracy (lol)

normalized_data.columns = ['1', '2', '3', '4']

normalized_data['4'][1] = 0.01

normalized_data.columns = [r - 1, '', '', '']

normalized_data

,9732,,,
0,,,,
1,H,-0.517051,0.0781024,0.01
2,H,-0.846049,0.919888,0
3,H,-0.268364,-0.0697354,0
4,H,1.56051,2.90878,0
...,...,...,...,...
9728,H,0.94042,-0.501382,0
9729,H,0.291682,-0.698664,0
9730,H,1.10755,3.69246,0
9731,H,0.471692,-0.228451,0


In [5]:
# Found here: https://stackoverflow.com/questions/41428539/data-frame-to-file-txt-python/41428596

normalized_data.to_csv(str(file_path) + '_reformatted.txt', index = False, header = True, sep = '\t', mode = 'a')